In [1]:
import backend
import json
import generate_page as page
initial_prompt = "Create a schema for a very simple twitter application"
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'


In [ ]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

In [ ]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

In [ ]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
actions = backend.read(actions_path)
schema_page = backend.create_schema(schema, schema_page_path)
crud_page = backend.create_crud(schema, crud_path)
# faker_data = backend.create_faker_data_code(schema, 'generated/faker.ts')
actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

In [ ]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)

In [4]:
# alternative way to make page structure: simplified version
page.write(page_path, page.generate_component_list(schema_path, actions_path))

[
  {
    "url": "/",
    "title": "Home",
    "description": "The main timeline page showing tweets from users the logged-in user is following"
  },
  {
    "url": "/login",
    "title": "Login",
    "description": "The login page for users to sign in to their account"
  },
  {
    "url": "/register",
    "title": "Register",
    "description": "The registration page for new users to create an account"
  },
  {
    "url": "/profile/[username]",
    "title": "UserProfile",
    "description": "A user's profile page showing their bio, tweets, followers, and users they are following"
  },
  {
    "url": "/compose",
    "title": "ComposeTweet",
    "description": "A page for the logged-in user to compose and post a new tweet"
  }
]
{
  "pages": [
    {
      "url": "/",
      "title": "Home",
      "description": "The main timeline page showing tweets from users the logged-in user is following",
      "components": [
        {
          "name": "Navbar",
          "description": "Navigatio

In [12]:
# generate single page
pages = json.loads(backend.read(page_path))["pages"]
page.generate_page(pages, pages[0], schema_path, actions_path)

Writing the Navbar component... (enter)
Here is the edited file for the Navbar component of the Home page:

<replace start=0 end=1>
'use client'
import Link from 'next/link'
import { useUser } from '@clerk/clerk-react'
</replace>

<replace start=3 end=5>
interface Props {
  userId: string | null
}
</replace>

<replace start=6 end=9>
export default ({ userId }: Props) => {
  const { isSignedIn, user } = useUser()

  return (
    <nav className="flex justify-between items-center p-4 bg-white shadow-md">
      <Link href="/">
        <h1 className="text-2xl font-bold">Twitter Clone</h1>
      </Link>
      <div className="flex items-center space-x-4">
        {isSignedIn ? (
          <>
            <Link href={`/profile/${user?.username}`}>
              <img
                src={user?.profileImageUrl}
                alt={user?.username}
                className="w-8 h-8 rounded-full"
              />
            </Link>
            <Link href="/compose" className="bg-blue-500 text-whi